In [51]:
import pandas as pd
import boto3
from io import StringIO 

# if you want to import a function from a python script into your notebooks, carry out this following steps

import os, sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.helpers import identify_missing_data

In [4]:
# step 1- reading the user data final file

users_data = pd.read_pickle("../../week11/data/users_data_final.pkl")

In [5]:
users_data.head()

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.0,2143.0,58,management,married,tertiary,False,True,False,None,261,1,-1,0,None,False,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,NaN,NaN,44,technician,single,secondary,False,True,False,None,151,1,-1,0,None,False,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.0,2.0,33,entrepreneur,married,secondary,False,True,True,None,76,1,-1,0,None,False,2002-06-04,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,NaN,NaN,47,blue-collar,married,None,False,True,False,None,92,1,-1,0,None,False,1995-06-29,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.0,1.0,33,None,single,None,False,False,False,None,198,1,-1,0,None,False,1995-08-01,mobile


In [6]:
users_data.shape

(45209, 19)

In [7]:
users_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45209 entries, 0 to 45215
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              45209 non-null  object        
 1   number_transactions  35210 non-null  float64       
 2   total_amount_usd     35210 non-null  float64       
 3   age                  45209 non-null  int64         
 4   job                  44921 non-null  object        
 5   marital              45209 non-null  object        
 6   education            43352 non-null  object        
 7   default              45209 non-null  bool          
 8   housing              45209 non-null  bool          
 9   loan                 45209 non-null  bool          
 10  contact              32191 non-null  object        
 11  duration             45209 non-null  int64         
 12  campaign             45209 non-null  int64         
 13  pdays                45209 non-null 

In [8]:
# question: how many columns have the data type object?

list(users_data.select_dtypes(['object']).columns)

['user_id', 'job', 'marital', 'education', 'contact', 'poutcome', 'device']

In [9]:
len(users_data.select_dtypes(['object']).columns)

7

In [10]:
print("The number of columns with data type object is", len(users_data.select_dtypes(['object']).columns))

The number of columns with data type object is 7


In [11]:
# question: how many columns have the data type int, float and bool?

list(users_data.select_dtypes(['int64', 'float64', 'bool']).columns)

['number_transactions',
 'total_amount_usd',
 'age',
 'default',
 'housing',
 'loan',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'term_deposit']

In [12]:
len(list(users_data.select_dtypes(['int64', 'float64', 'bool']).columns))

11

In [13]:
print("The number of columns with data type object is", len(list(users_data.select_dtypes(['int64', 'float64', 'bool']).columns)))

The number of columns with data type object is 11


In [14]:
# question: were there are duplicate rows of data?

users_data[users_data.duplicated()]

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device


In [15]:
users_data.duplicated().sum()

np.int64(0)

In [16]:
users_data.isnull().sum()

user_id                    0
number_transactions     9999
total_amount_usd        9999
age                        0
job                      288
marital                    0
education               1857
default                    0
housing                    0
loan                       0
contact                13018
duration                   0
campaign                   0
pdays                      0
previous                   0
poutcome               36957
term_deposit               0
date_joined               30
device                    94
dtype: int64

In [17]:
users_data.isnull().mean()

user_id                0.000000
number_transactions    0.221173
total_amount_usd       0.221173
age                    0.000000
job                    0.006370
marital                0.000000
education              0.041076
default                0.000000
housing                0.000000
loan                   0.000000
contact                0.287952
duration               0.000000
campaign               0.000000
pdays                  0.000000
previous               0.000000
poutcome               0.817470
term_deposit           0.000000
date_joined            0.000664
device                 0.002079
dtype: float64

In [18]:
users_data[users_data.poutcome.isnull()]

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.0,2143.0,58,management,married,tertiary,False,True,False,None,261,1,-1,0,None,False,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,NaN,NaN,44,technician,single,secondary,False,True,False,None,151,1,-1,0,None,False,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.0,2.0,33,entrepreneur,married,secondary,False,True,True,None,76,1,-1,0,None,False,2002-06-04,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,NaN,NaN,47,blue-collar,married,None,False,True,False,None,92,1,-1,0,None,False,1995-06-29,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.0,1.0,33,None,single,None,False,False,False,None,198,1,-1,0,None,False,1995-08-01,mobile
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45208,240bce69-b3eb-49c6-9a61-f0684b919d7a,2.0,113.0,23,student,single,tertiary,False,False,False,cellular,266,1,-1,0,None,True,1997-02-09,desktop
45210,6ace2eec-d1a9-4ea6-810d-4acb56fd00ab,3.0,505.0,25,technician,single,secondary,False,False,True,cellular,386,2,-1,0,None,True,2007-08-30,tablet
45211,a218c1d6-880a-4431-a314-a1bd3dc36b21,2.0,825.0,51,technician,married,tertiary,False,False,False,cellular,977,3,-1,0,None,True,2002-12-28,desktop
45212,6a6bf402-9416-4e71-97c6-727e8298a3a5,4.0,1729.0,71,retired,divorced,primary,False,False,False,cellular,456,2,-1,0,None,True,2010-11-18,desktop


In [23]:
missing_data = identify_missing_data(users_data)

In [24]:
missing_data

,variable,percent_missing,data_type
0,poutcome,81.75,object
1,contact,28.80,object
2,number_transactions,22.12,float64
3,total_amount_usd,22.12,float64
4,education,4.11,object
5,job,0.64,object
6,device,0.21,object
7,date_joined,0.07,datetime64[ns]


In [25]:
# question: how many columns contain missing data

# option 1 
missing_data.shape[0]

8

In [26]:
len(missing_data)

8

In [27]:
print("# of columns with missing data", missing_data.shape[0])

# of columns with missing data 8


In [28]:
missing_data[missing_data.data_type == 'float64']

,variable,percent_missing,data_type
2,number_transactions,22.12,float64
3,total_amount_usd,22.12,float64


In [29]:
missing_data[missing_data.data_type == 'float64'].shape[0]

2

In [30]:
print("# of columns with missing data and data type float:", missing_data[missing_data.data_type == 'float64'].shape[0])

# of columns with missing data and data type float: 2


In [31]:
# how many columns contain missing data and have the data type object??

missing_data[missing_data.data_type == 'object']

,variable,percent_missing,data_type
0,poutcome,81.75,object
1,contact,28.80,object
4,education,4.11,object
5,job,0.64,object
6,device,0.21,object


In [32]:
missing_data[missing_data.data_type == 'object'].shape[0]

5

In [33]:
print("# of columns with missing data and data type object:", missing_data[missing_data.data_type == 'object'].shape[0])

# of columns with missing data and data type object: 5


In [34]:
# question: identify the column that has more than 70% missing data

list(missing_data[missing_data.percent_missing > 70].variable)

['poutcome']

In [35]:
len(list(missing_data[missing_data.percent_missing > 70].variable))

1

In [36]:
# transformation task - drop the columns with more than 70% missing data
# option 1
users_data_transformed = users_data.drop(['poutcome'], axis=1)

In [66]:
# option 2

cols_to_drop = list(missing_data[missing_data.percent_missing > 70].variable)
cols_to_drop

['poutcome']

In [67]:
users_data_transformed = users_data.drop(cols_to_drop, axis=1)

In [68]:
identify_missing_data(users_data_transformed)

,variable,percent_missing,data_type
0,contact,28.80,object
1,number_transactions,22.12,float64
2,total_amount_usd,22.12,float64
3,education,4.11,object
4,job,0.64,object
5,device,0.21,object
6,date_joined,0.07,datetime64[ns]


In [69]:
print(users_data.shape)
print(users_data_transformed.shape)

(45209, 19)
(45209, 18)


In [38]:
# looking at where date joined is missing

users_data_transformed[users_data_transformed.date_joined.isnull()]

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,term_deposit,date_joined,device
1832,a8efc367-e7bc-4ddf-a075-ab6a850c5be1,2.0,10.0,40,blue-collar,married,secondary,False,True,False,None,1692,2,-1,0,True,NaT,desktop
1970,e287579c-0eb5-4da0-a697-a92e6a024c47,6.0,391.0,34,admin.,single,secondary,False,True,False,None,192,2,-1,0,False,NaT,mobile
8033,9e6d7f37-fec1-4967-9281-d37eb69fc211,4.0,691.0,34,technician,married,tertiary,False,True,True,None,70,1,-1,0,False,NaT,mobile
11292,e380ff78-00d5-4f4d-93a3-db991ff0ca3b,3.0,2769.0,41,services,divorced,secondary,False,True,False,None,96,3,-1,0,False,NaT,mobile
12439,d64a3876-1dc0-480e-b13e-c4b89639b6d2,5.0,0.0,57,retired,married,secondary,False,False,True,None,104,2,-1,0,False,NaT,mobile
13006,2673ec0e-5df6-416f-9eda-e39cb50ca03c,2.0,473.0,32,services,single,secondary,False,True,False,cellular,624,5,-1,0,False,NaT,mobile
13826,a0e4abc2-6451-4a97-a669-45307de8059d,2.0,29.0,57,retired,married,tertiary,False,False,False,telephone,532,2,-1,0,False,NaT,mobile
15175,1101e360-4872-44a9-a3ac-38afa2f73581,NaN,NaN,33,self-employed,divorced,secondary,False,True,False,cellular,1584,2,-1,0,True,NaT,mobile
15290,a63e7d7d-00ba-4751-be20-440576efaac3,2.0,976.0,42,admin.,married,secondary,False,True,False,cellular,69,7,-1,0,False,NaT,mobile
17032,e5b0bc16-b7fc-43a8-9d4f-90827fbe8f1f,NaN,NaN,51,unemployed,married,primary,False,True,False,cellular,2015,1,-1,0,True,NaT,mobile


In [39]:
users_data_transformed[users_data_transformed.date_joined.isnull()].shape[0]

30

In [40]:
# 

users_data_final = users_data_transformed[~users_data_transformed.date_joined.isnull()]

In [43]:
print("Original data set:", users_data.shape)
print("Size of data after dropping columns with more than 70% missing data:", users_data_transformed.shape)
print("Size of data after dropping rows where date joined is missing:", users_data_final.shape)


Original data set: (45209, 19)
Size of data after dropping columns with more than 70% missing data: (45209, 18)
Size of data after dropping rows where date joined is missing: (45179, 18)


In [44]:
identify_missing_data(users_data_final)

,variable,percent_missing,data_type
0,contact,28.80,object
1,number_transactions,22.11,float64
2,total_amount_usd,22.11,float64
3,education,4.11,object
4,job,0.64,object
5,device,0.21,object


In [45]:
# saving our dataframe to a location

users_data_final.to_pickle("../data/users_data_transformed.pkl")

In [46]:
check = pd.read_pickle("../data/users_data_transformed.pkl")
check.head()

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.0,2143.0,58,management,married,tertiary,False,True,False,None,261,1,-1,0,False,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,NaN,NaN,44,technician,single,secondary,False,True,False,None,151,1,-1,0,False,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.0,2.0,33,entrepreneur,married,secondary,False,True,True,None,76,1,-1,0,False,2002-06-04,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,NaN,NaN,47,blue-collar,married,None,False,True,False,None,92,1,-1,0,False,1995-06-29,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.0,1.0,33,None,single,None,False,False,False,None,198,1,-1,0,False,1995-08-01,mobile


In [47]:
check.shape

(45179, 18)

In [63]:
# to load the data to s3
# step 1. connect to s3

# you need to create variables to store the connection info

aws_access_key = ""
aws_secret_access_key = ""
aws_s3_bucket = "waia-march-bootcamp"


In [ ]:
! pip3 install boto3

In [69]:
# make connection to s3

s3_conn = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_access_key
)

In [70]:
s3_conn

In [71]:
# load the file to s3

key = "transformations/sh_user_data_final.csv"

csv_buffer = StringIO() # create buffer to temporarily store the Data Frame
users_data_final.to_csv(csv_buffer, index=False) # code to write the data frame as csv file
response = s3_conn.put_object(
        Bucket=aws_s3_bucket, Key=key, Body=csv_buffer.getvalue()
    ) # this code writes the temp stored csv file and writes to s3


In [86]:
# you can also read a data file from s3

key = "transformations/la_user_data_final.csv" 

response = s3_conn.get_object(Bucket=aws_s3_bucket,
                   Key=key)
student_file = pd.read_csv(response.get("Body"))

In [87]:
student_file.shape

(45179, 18)

In [88]:
student_file.head()

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.0,2143.0,58,management,married,tertiary,False,True,False,Unknown,261,1,-1,0,False,1998-08-23,Unknown
1,bb92765a-08de-4963-b432-496524b39157,0.0,0.0,44,technician,single,secondary,False,True,False,Unknown,151,1,-1,0,False,2008-07-15,Unknown
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.0,2.0,33,entrepreneur,married,secondary,False,True,True,Unknown,76,1,-1,0,False,2002-06-04,Unknown
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,0.0,0.0,47,blue-collar,married,Unknown,False,True,False,Unknown,92,1,-1,0,False,1995-06-29,Unknown
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.0,1.0,33,Unknown,single,Unknown,False,False,False,Unknown,198,1,-1,0,False,1995-08-01,Unknown


In [89]:
student_file.isnull().sum()

user_id                0
number_transactions    0
total_amount_usd       0
age                    0
job                    0
marital                0
education              0
default                0
housing                0
loan                   0
contact                0
duration               0
campaign               0
pdays                  0
previous               0
term_deposit           0
date_joined            0
device                 0
dtype: int64

In [90]:
# create a definition to connect to s3

def connect_to_s3(aws_access_key_id, aws_secret_access_key):
    """Methods that connects to s3"""

    s3_client = boto3.client(
        "s3",
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key
    )

    print("Connection to s3 made")

    return s3_client

In [99]:
def load_df_to_s3(df, key, s3_bucket, aws_access_key_id, aws_secret_access_key):
    """Function that writes a data frame as a .csv file to a s3 bucket"""
    
    s3_client = connect_to_s3(aws_access_key_id, aws_secret_access_key)
    
    csv_buffer = StringIO() # create buffer to temporarily store the Data Frame
    df.to_csv(csv_buffer, index=False) # code to write the data frame as csv file
    response = s3_client.put_object(
            Bucket=s3_bucket, Key=key, Body=csv_buffer.getvalue()
        ) # this code writes the temp stored csv file and writes to s3

    print(f"The transformed data in the following location s3://{s3_bucket}/{key}")

In [101]:
key = "transformations2/sh_user_data_final.csv"

load_df_to_s3(users_data_final, key, aws_s3_bucket, aws_access_key, aws_secret_access_key)

Connection to s3 made
The transformed data in the following location s3://waia-march-bootcamp/transformations2/sh_user_data_final.csv
